In [ ]:
from astropy import constants as cnst   # Get constant in astronomy
from datetime import datetime           # current time
from tqdm import tqdm                   # progress bar 
from PIL import Image                   # make gif
from scipy.interpolate import interp1d  # distance fitting about density function

import matplotlib.pyplot as plt         # make graph
import numpy as np                      
import os                               #access to system directory

In [ ]:
#take current time
now = datetime.now()
current = now.strftime('%m_%d_%H_%M')
print(current)

In [ ]:
#set constant value
GRAV = 6.6743e-11                                   # Gravitational constant
M_SUN = 1.98892e30                                  # Solar mass / kg
KPC = 3.0856776e+19                                 # Kiloparsec / m
MYR = 24.0 * 3600 * 365.2526 * 1e6                  # Megayear / s
SMBH = M_SUN * 1.7e8

#Simulation setting
N_TOT = 5000                                        # Numer of total objects

R0 = [0, 0]                                                 # Galaxy location
R = 46.56 / 2 * KPC                                         # Galaxy Raidus
M_TOT = 1.5e12 * M_SUN                                      # Galaxy Mass
M_BARYON = (1.3e11 + 7.2e9 + 3.4e8 + 5.4e7) * M_SUN         # Mass of observable matters (except SMBH)

N_BULGE = int(N_TOT * 0.1)
R_BULGE = 1.5 * KPC                                     # radius of bulge ~ 1.5kpc
V_VAR_bulge = 160                                       # velocity variance of bulge ~160km/s
M_BULGE_total = M_BARYON * 0.4     
M_BULGE_each = M_BULGE_total / N_BULGE

N_DISC = int(N_TOT * 0.3)
M_DISC_total = M_BARYON - M_BULGE_total
M_DISC_each = M_DISC_total / N_DISC

N_HALO = N_TOT - N_BULGE - N_DISC
R_HALO = 67.45 / 2 * KPC
# R_HALO = 306.6 * KPC
M_HALO_total = M_TOT - M_BARYON
M_HALO_each = M_HALO_total / N_HALO

softening_parameter = 2e20
softening_sq = softening_parameter ** 2

def softening(dist_sq, softening_sq):
    return softening_sq * softening_sq / (dist_sq + softening_sq)

dt = MYR * 3                                       # Time interval between frame
frames = 500                                        # Number of frames

directory = '/home/ddhy/codes/n_body_galaxy_interaction/'

In [ ]:
def v_rotation(distance, dist_x, dist_y, v_0=2.5e5, r_0=10 * KPC):
    v_r = v_0 * (1 - np.exp(-5 * distance/r_0))
    v_x = v_r * dist_y / distance * -1    # CCW
    v_y = v_r * dist_x / distance
    return v_x, v_y

# Density function of DM (Burkert profile)
def density_func(r, rho_0, r_0):
    return (rho_0 * r_0**3) / ((r + r_0) * (r**2 + r_0**2))

# Distribution of DM particles using density_func
def cumulative_number_density(r_vals):
    r_0 = 10 * KPC
    rho_0 = M_HALO_total / np.pi / R_HALO / R_HALO
    dr_vals = np.diff(r_vals)  # distance interval
    num_density_elements = density_func(r_vals[:-1], rho_0, r_0) * 4 * np.pi * r_vals[:-1]**2 * dr_vals  # particle number of each part
    cumulative_density = np.cumsum(num_density_elements)
    cumulative_density = np.insert(cumulative_density, 0, 0)  # list of weight particle
    return cumulative_density / cumulative_density[-1]  # return after normalization

def gen_stars():
    # Initial star location
    angles_bulge = np.linspace(0, 2446 * np.pi, N_BULGE)
    angles_disc  = np.linspace(0, 2446 * np.pi, N_DISC)
    angles_halo  = np.linspace(0, 2446 * np.pi, N_HALO)

    angles = np.concatenate((angles_bulge, angles_disc, angles_halo))

    # rand_r = np.random.uniform(0, R, N_TOT)
    rand_r_bulge = np.random.uniform(low=1e-5, high=1, size=N_BULGE) * R_BULGE
    rand_r_disc  = np.random.uniform(low=R_BULGE/R, high=1, size=N_DISC) * R
    rand_r_halo  = np.random.uniform(low=1e-5, high=1, size=N_HALO) * R_HALO

    cumulative_number = cumulative_number_density(rand_r_halo)
    inv_cdf = interp1d(cumulative_number, rand_r_halo, kind='linear')
    r_halo = inv_cdf(np.linspace(0, 1, N_HALO))

    rand_r = np.concatenate((rand_r_bulge, rand_r_disc, r_halo))

    # Convert polar coordinates to Cartesian coordinates
    sx = rand_r * np.cos(angles)
    sy = rand_r * np.sin(angles)

    vx, vy = v_rotation(rand_r, sx, sy)

    coordinate = np.array([sx, sy])
    velocity = np.array([vx, vy])
    velocity[0][0] = 0
    velocity[1][0] = 0

    mass_bulge = np.ones(N_BULGE) * M_BULGE_each
    mass_disc = np.ones(N_DISC) * M_DISC_each
    mass_halo = np.ones(N_HALO) * M_HALO_each

    mass = np.concatenate((mass_bulge, mass_disc, mass_halo))

    # Set SMBH in the center
    mass[0] = SMBH
    # mass[0] = M_SUN
    sx[0], sy[0] = 0, 0  # Place SMBH at the center of the galaxy

    return coordinate, velocity, mass


In [ ]:
coordinate, velocity, mass = gen_stars()

plt.hist(mass)
plt.show()

In [ ]:
# calculate distance and force
def compute_accelerations(coords, mass):
    dx = coords[0, :, None] - coords[0, None, :]
    dy = coords[1, :, None] - coords[1, None, :]
    dist_sq = dx**2 + dy**2
    dist = np.sqrt(dist_sq + softening(dist_sq, softening_sq))

    # gravitaional acceleration
    force = -GRAV * mass[None, :] / dist**3
    accel_x = np.sum(force * dx, axis=1)
    accel_y = np.sum(force * dy, axis=1)
    return np.array([accel_x, accel_y])

def force_rk4(coordinate, velocity, mass, dt):

    # copy initial condition
    coords = np.copy(coordinate)
    vels = np.copy(velocity)

    # calculate k1 
    accel_k1 = compute_accelerations(coords, mass)
    k1_vel = accel_k1 * dt
    k1_pos = vels * dt

    # calculate k2
    coords_k2 = coords + 0.5 * k1_pos
    vels_k2 = vels + 0.5 * k1_vel
    accel_k2 = compute_accelerations(coords_k2, mass)
    k2_vel = accel_k2 * dt
    k2_pos = vels_k2 * dt

    # calculate k3
    coords_k3 = coords + 0.5 * k2_pos
    vels_k3 = vels + 0.5 * k2_vel
    accel_k3 = compute_accelerations(coords_k3, mass)
    k3_vel = accel_k3 * dt
    k3_pos = vels_k3 * dt

    # calculate k4
    coords_k4 = coords + k3_pos
    vels_k4 = vels + k3_vel
    accel_k4 = compute_accelerations(coords_k4, mass)
    k4_vel = accel_k4 * dt
    k4_pos = vels_k4 * dt

    # final calculate
    velocity += (k1_vel + 2 * k2_vel + 2 * k3_vel + k4_vel) / 6
    coordinate += (k1_pos + 2 * k2_pos + 2 * k3_pos + k4_pos) / 6

    return coordinate, velocity

In [ ]:
def move():
    global coordinate, velocity  # set to global

    # set SMBH location
    coordinate[0][0], coordinate[1][0] = 0, 0
    velocity[0][0], velocity[1][0] = 0, 0

    #apply Rk4
    coordinate, velocity = force_rk4(coordinate, velocity, mass, dt)


In [ ]:
# n-body simulation
def nbody():

    gif_dir = directory+'gifs'
    each_images_dir = directory+f'images/n_body_{current}'
    
    os.makedirs(each_images_dir, exist_ok=True)
    os.makedirs(gif_dir, exist_ok=True)

    print('Initial position')
    fig, ax = plt.subplots(figsize=(10, 10))

    #set SMBH size in plot
    sizes = np.ones_like(coordinate[0]) * 1  # basic size
    sizes[0] = 300  # coordinate[0] set SMBH size as 300

    coordinate[0][0], coordinate[1][0] = 0, 0   #set SMBH location

    # initial position setting
    ax.scatter(
        coordinate[0][1:N_TOT-N_HALO],
        coordinate[1][1:N_TOT-N_HALO],
        s=sizes[1:N_TOT-N_HALO],
        c='magenta',
        alpha=0.7
    )
    ax.scatter(
        coordinate[0][N_TOT-N_HALO:],
        coordinate[1][N_TOT-N_HALO:],
        s=sizes[N_TOT-N_HALO:],
        c='black',
        alpha=0.2
    )
    ax.scatter(
        coordinate[0][0],
        coordinate[1][0],
        s=sizes[0],
        c='blue',
        alpha=0.6
    )
    ax.plot([],[],label=f'N = {N_TOT}')
    ax.plot([],[],label=f'dt = {dt/MYR:.1f} Myr')
    ax.plot([],[],label=f'R = {R/KPC:.1f} kpc')
    ax.plot([],[],label=f'softening = {softening_parameter:.1e}')
    ax.legend(loc='lower right')
    ax.set_xlim(-R*2, R*2)
    ax.set_ylim(-R*2, R*2)
    ax.set_aspect('equal')

    plt.show()
    plt.close()

    print(f'\nCalculating {N_TOT}-body gravitational interaction\n')

    # calculate motion
    png_files = []
    with tqdm(total=frames) as pbar:  # progress bar
        for t in range(frames):

            move()  # mition

            fig, ax = plt.subplots(figsize=(10, 10))
            
            ax.scatter(
                coordinate[0][1:N_TOT-N_HALO],
                coordinate[1][1:N_TOT-N_HALO],
                s=sizes[1:N_TOT-N_HALO],
                c='magenta',
                alpha=0.5
            )
            ax.scatter(
                coordinate[0][N_TOT-N_HALO:],
                coordinate[1][N_TOT-N_HALO:],
                s=sizes[N_TOT-N_HALO:],
                c='black',
                alpha=0.2
            )
            ax.scatter(
                coordinate[0][0],
                coordinate[1][0],
                s=sizes[0],
                c='blue',
                alpha=0.6
            )
            ax.plot([],[],label=f'N = {N_TOT}')
            ax.plot([],[],label=f'dt = {dt/MYR:.1f} Myr')
            ax.plot([],[],label=f'R = {R/KPC:.1f} kpc')
            ax.plot([],[],label=f'softening parameter = {softening_parameter:.3e}')
            ax.legend(loc='lower right')
            ax.set_xlim(-R*2, R*2)
            ax.set_ylim(-R*2, R*2)
            ax.set_aspect('equal')
            ax.set_title(f'image_{t:03d}')

            frame_path = os.path.join(each_images_dir, f'frame_{current}_{t:04d}.png')
            fig.savefig(frame_path)
            png_files.append(frame_path)

            plt.close()
            pbar.update(1)
    
    print('\nConverting PNG to GIF\n')

    # PNG to GIF
    with Image.open(png_files[0]) as img:
        img.save(
            os.path.join(gif_dir, f'n_body_{current}.gif'),
            save_all=True,
            append_images=[Image.open(f) for f in png_files[1:]],
            duration=40,
            loop=0,
        )
        
    print('Complete!!\n')


In [ ]:
nbody()